In [2]:
import json
import pandas as pd
import re
## Training set preperation
dict_mapping = {'Presenting Irrelevant Data (Red Herring)': 0,
 'Bandwagon': 1,
 'Smears': 2,
 'Glittering generalities (Virtue)': 3,
 'Causal Oversimplification': 4,
 'Whataboutism': 5,
 'Loaded Language': 6,
 'Exaggeration/Minimisation': 7,
 'Repetition': 8,
 'Thought-terminating cliché': 9,
 'Name calling/Labeling': 10,
 'Appeal to authority': 11,
 'Black-and-white Fallacy/Dictatorship': 12,
 'Obfuscation, Intentional vagueness, Confusion': 13,
 'Reductio ad hitlerum': 14,
 'Appeal to fear/prejudice': 15,
 "Misrepresentation of Someone's Position (Straw Man)": 16,
 'Flag-waving': 17,
 'Slogans': 18,
 'Doubt': 19,
 'Bandwagon_1': 20,
 'Whataboutism_1': 21,
 'Appeal to fear/prejudice_1': 22,
 'Flag-waving_1': 23,
 'Appeal to authority_1': 24,
 "Transfer_1" : 25,
 "Transfer" : 26,
 "Appeal to (Strong) Emotions" : 27
}


def dataset_f(data,dict_mapping):
    set_x = set()
    for element in data:
        for label in element['labels']:
            set_x.add(label)
    
    data_id = []
    for element in data:
        dict_x = {}
        dict_x["text"] = element["text"]
        dict_x["labels"] = element["labels"]
        dict_x["id"] = element["id"]
        dict_x["image"] = element["image"]
        if len(element['labels']):
            data_id.append(dict_x)
    
    unprocessed = []
    processed = []
    labels = []
    ID = []
    image_ID = []
    for element in data_id:
        cleaned_text = re.sub(r'&', 'and',element['text'])
        pattern = r'\\n'
        cleaned_text = re.sub(pattern, ' ',cleaned_text)
        pattern = r'[^a-zA-Z0-9\s]+'
        cleaned_text = re.sub(pattern, '', cleaned_text).lower()
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
        if(cleaned_text != ''):
            unprocessed.append(element['text'])
            processed.append(cleaned_text)
            labels.append(element['labels'])
            ID.append(element['id'])
            image_ID.append(element['image'])
            
        
    data = {'text': unprocessed, 'aug_text': processed, 'label': labels,'id':ID,'image_ID' : image_ID}
    df = pd.DataFrame(data)

    for index,row in df.iterrows():
        l_labels = row['label']
        empty_list = []
        for label in l_labels:
            empty_list.append(dict_mapping[label])
        row['label'] = empty_list

    return df
    
with open(r'train.json', 'rb') as json_file:
    data = json.load(json_file)

df_train = dataset_f(data,dict_mapping)

with open(r'validation.json', 'rb') as json_file:
    data = json.load(json_file)

df_val = dataset_f(data,dict_mapping)


df_train_full = pd.concat([df_train,df_val],axis=0)
df_exploded_train = df_train_full.explode('label')

condition = df_exploded_train['label'].isin([1, 5, 15, 17, 11, 26])
df_duplicated = df_exploded_train[condition].copy()
mapping_dict = {1: 20, 5: 21, 15: 22, 17: 23, 11: 24,26 : 25}
df_duplicated['label'] = df_duplicated['label'].map(mapping_dict)
df_duplicated = df_duplicated.drop_duplicates()
result_df = pd.concat([df_exploded_train, df_duplicated], ignore_index=True)
result_df.to_csv('Merged_train_for_HypEmo_2b.csv')

In [3]:

def dataset_f(data,dict_mapping):
    data_id = []
    for element in data:
        dict_x = {}
        dict_x["text"] = element["text"]
        dict_x["id"] = element["id"]
        dict_x["image"] = element["image"]
        data_id.append(dict_x)
        
    unprocessed = []
    processed = []
    labels = []
    ID = []
    image_ID = []
    
    print(len(data_id))
    for element in data_id:
        unprocessed.append(element['text'])
        cleaned_text = re.sub(r'&', 'and',element['text'])
        cleaned_text = cleaned_text.lower()
        processed.append(cleaned_text)
        x = 9
        labels.append(x)
        ID.append(element['id'])
        image_ID.append(element['image'])
    
    data = {'text': unprocessed, 'aug_text': processed, 'label': labels,'id':ID,'image_ID' : image_ID}
    df = pd.DataFrame(data)
    return df

# with open(r'data_given_to_work_with/subtask1/dev_subtask1_en.json', 'rb') as json_file:
#     data = json.load(json_file)
# df_dev_eng_full = dataset_f(data,dict_mapping)

with open(r'test_data/en_subtask2a_test_unlabeled.json', 'rb') as json_file:
    data = json.load(json_file)
df_test_eng_full = dataset_f(data,dict_mapping)


1500


In [4]:
from transformers import MarianMTModel, MarianTokenizer
from tqdm import tqdm
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with open('test_data/ar_subtask2a_test_unlabeled.json', 'rb') as json_file:
    data = json.load(json_file)
df_test_arab =  dataset_f(data,dict_mapping)
src_text = list(df_test_arab["text"])
model_name = "Helsinki-NLP/opus-mt-ar-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to(device)
t_text = []
batch_size = 16
num_samples = len(src_text)
translations = []
for i in tqdm(range(0, num_samples, batch_size)):
    batch_src_text = src_text[i:i + batch_size]
    batch_src_text = tokenizer(batch_src_text, return_tensors="pt", padding=True, truncation=True).to(device)
    batch_translated = model.generate(**batch_src_text).to("cpu")
    translations.extend(batch_translated)
for t in tqdm(translations):
    t_text.append(tokenizer.decode(t, skip_special_tokens=True))    
df_test_arab["aug_text"] = t_text


120


  0%|                                                                                            | 0/8 [00:00<?, ?it/s]C:\Users\tejas\anaconda3\envs\HypEmo\lib\site-packages\transformers\generation\utils.py:1353: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████| 120/120 [00:00<00:00, 1249.58it/s]


In [5]:
df_test_arab

,text,aug_text,label,id,image_ID
0,مشايخ الأوقاف واقفين لحماية المسجد\nحماية المس...,The Waqf Chiefs are standing to protect the mo...,9,00001,prop_meme_00001.jpg
1,المسؤولين العرب بعدما يخرو على شعبهم و حان الآ...,"Arab officials, after they've fallen on their ...",9,00002,prop_meme_00002.jpg
2,عندما يسأل بوتفليقة عن المدة التي يريد قضاءها ...,When Pottflika asks how long Renessa wants to ...,9,00003,prop_meme_00003.jpg
3,لا يعني انه اذا حطيت ايدي عليكي او اتطلعت ع صد...,Doesn't mean that if I put Eddie on you or loo...,9,00004,prop_meme_00004.jpg
4,*اسرائيل*\n*قتل وهدم واعتقالات\n* مستوطنات\n*ض...,"Israel's killing, demolition and arrests of se...",9,00005,prop_meme_00005.jpg
...,...,...,...,...,...
115,هذا القط حياته بائسه جدا وفقيرلايملك ربع تونه💔\n,"This cat's life is so miserable and poor, he d...",9,00499,prop_meme_00499.jpg
116,| هى كورونا دى داجل ولا ست\n والله يابنتى طالم...,"It's Corona de Doughle and La Six, and God, my...",9,00506,prop_meme_00506.jpg
117,بتضرب أعز أصحابك مقابل مليون دولار؟؟\nأنا:,By hitting your best friend for a million doll...,9,00541,prop_meme_00541.jpg
118,ولادي بعد ما تنمرت على نص الكوكب,My birth after you bullied on the planet's text.,9,00542,prop_meme_00542.jpg


In [6]:
from transformers import MarianMTModel, MarianTokenizer
from tqdm import tqdm
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
with open('test_data/bg_subtask2a_test_unlabeled.json', 'rb') as json_file:
    data = json.load(json_file)
df_test_bulg =  dataset_f(data,dict_mapping)
src_text = list(df_test_bulg["text"])
model_name = "Helsinki-NLP/opus-mt-bg-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to(device)
t_text = []
batch_size = 8
num_samples = len(src_text)
translations = []
for i in tqdm(range(0, num_samples, batch_size)):
    batch_src_text = src_text[i:i + batch_size]
    batch_src_text = tokenizer(batch_src_text, return_tensors="pt", padding=True, truncation=True).to(device)
    batch_translated = model.generate(**batch_src_text).to("cpu")
    translations.extend(batch_translated)
for t in tqdm(translations):
    t_text.append(tokenizer.decode(t, skip_special_tokens=True).lower())    
df_test_bulg["aug_text"] = t_text

436


  0%|                                                                                           | 0/55 [00:00<?, ?it/s]C:\Users\tejas\anaconda3\envs\HypEmo\lib\site-packages\transformers\generation\utils.py:1353: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████| 436/436 [00:00<00:00, 1113.09it/s]


In [7]:
df_test_bulg

,text,aug_text,label,id,image_ID
0,ДНЕВНИК\n\nКАПИТАЛ\n\nAMERICA FOR BULGARIA FOU...,derivative tier 1 america for bulgaria foundat...,9,bg_mk_memes_83,bg_mk_memes_83.png
1,Когато съм се ваксинирал и се върна на село да...,when i was vaccinated and returned to the vill...,9,bg_mk_memes_104,bg_mk_memes_104.png
2,"Ганя, ваксинирай се, глупако!!\n\nГаня, полудя...","ganya, you're inoculating yourself, you idiot!",9,bg_mk_memes_106,bg_mk_memes_106.png
3,Не можеш ли да работиш нещо нормално като друг...,can't you work something normal like other peo...,9,bg_mk_memes_148,bg_mk_memes_148.png
4,Не назеления сертификат\n\nВаксиниран съм\nПри...,no green certificate i've been vaccinated i've...,9,bg_mk_memes_237,bg_mk_memes_237.png
...,...,...,...,...,...
431,@MEME PARTIQ\nМакедонска\nСветовна\nистория\nи...,@mememe partiq macedonian world history,9,74,bg_memes_74.png
432,"Никога не е била!\nЧакай, Земята не\nе македон...","wait, the earth isn't macedonian?",9,75,bg_memes_75.png
433,Тате Тате || даа\nАз съм\nМакедонфц\nthe\nЩе г...,"dad, i'm the macedonfz, you're gonna burn at y...",9,76,bg_memes_76.png
434,МАКЕДОНИЯ и\nБЪЛГАРИЯ\nМАКЕДОНИЯ и\n- БЪЛГАРИЯ...,macedonia and bulgaria macedonia and - bulgari...,9,8,bg_memes_8.png


In [9]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [10]:
from transformers import MarianMTModel, MarianTokenizer
from tqdm import tqdm
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
with open('test_data/mk_subtask2a_test_unlabeled.json', 'rb') as json_file:
    data = json.load(json_file)
    
df_test_mace =  dataset_f(data,dict_mapping)
src_text = list(df_test_mace["text"])
model_name = "Helsinki-NLP/opus-mt-mk-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to(device)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

t_text = []
batch_size = 16
num_samples = len(src_text)
translations = []

for i in tqdm(range(0, num_samples, batch_size)):
    batch_src_text = src_text[i:i + batch_size]
    batch_src_text = tokenizer(batch_src_text, return_tensors="pt", padding=True, truncation=True).to(device)
    batch_translated = model.generate(**batch_src_text).to("cpu")
    translations.extend(batch_translated)

for t in tqdm(translations):
    t_text.append(tokenizer.decode(t, skip_special_tokens=True).lower())

df_test_mace["aug_text"] = t_text

259


  0%|                                                                                           | 0/17 [00:00<?, ?it/s]C:\Users\tejas\anaconda3\envs\HypEmo\lib\site-packages\transformers\generation\utils.py:1353: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████| 259/259 [00:00<00:00, 1763.96it/s]


In [12]:
df_test_full = pd.concat([df_test_eng_full,df_test_arab,df_test_bulg,df_test_mace],ignore_index=True)

In [13]:
df_test_full
df_test_full.to_csv('Merged_test_for_HypEmo_2b.csv')

,text,aug_text,label,id,image_ID
0,Nicola Sturgeon\n\nWE'RE SCOTTISH GETUSOUTOFHE...,nicola sturgeon\n\nwe're scottish getusoutofhe...,9,68254,prop_meme_8883.png
1,I saw a movie once where only the police and m...,i saw a movie once where only the police and m...,9,69640,prop_meme_10818.png
2,Heaven has a Wall and strict immigration polic...,heaven has a wall and strict immigration polic...,9,71251,prop_meme_7048.png
3,Don't expect a broken government to fix itself.,don't expect a broken government to fix itself.,9,79369,prop_meme_15611.png
4,HOW MOST AMERICANS SEE THE DEBATE\n\nHOW FREE ...,how most americans see the debate\n\nhow free ...,9,69351,prop_meme_10392.png
...,...,...,...,...,...
2310,Невакцинираните\nсе потенцијални\nсамоубијци и...,invalids are potential suicides and will not e...,9,mk_memes_319,mk_memes_319.png
2311,И ВО ВТОРАТА СВЕТСКА ВОЈНА ИМАШЕ НАШИ ШТО ОДЕА...,"and in the second world war, ours was what wen...",9,mk_memes_321,mk_memes_321.png
2312,Методија Андонов\nЧенто бил калкулант.\nДури о...,"methodiusa andonov chento was a calculator, on...",9,mk_memes_323,mk_memes_323.png
2313,НЕМА ВЕЌЕ ДАТУМ\n\n,no more date,9,mk_memes_324,mk_memes_324.png
